In [ ]:
import sagemaker
from sagemaker.mxnet import MXNet
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [ ]:
# Download the data on local disk
from mxnet.gluon.data.vision import datasets
trainset = datasets.CIFAR10(root='./data', train=True)
testset = datasets.CIFAR10(root='./data', train=False)

In [ ]:
inputs = sagemaker_session.upload_data(path='data', key_prefix='data/dawnbench-gluon-cifar10')
print('input spec (in this case, just an S3 path): {}'.format(inputs))

In [ ]:
import os
source_dir = os.path.join(os.getcwd(), 'training_code')
train_instance_type="ml.p3.2xlarge"



### Configuration for the stable convergence. 
### This should get to 94% in around 35 epochs, taking a little more than 4 minutes.
hyperparameters={'batch_size': 512,
                 'lr': 0.20117947903692157,
                 'momentum': 0.8871656440409877,
                 'epochs': 31,
                 'extra': 7,
                 'wd': 0.0013947371011895455,
                 'peak': 9,
                }



### Configuration for the fastest job.
### This has converged in 30 epochs, taking about 3min47s. However the results are not as easily reproducible.
### Uncomment if you want to run this configuration.
# hyperparameters={'batch_size': 512,
#                  'lr': 0.3,
#                  'momentum': 0.8989643139985577,
#                  'epochs': 30,
#                  'extra': 7,
#                  'wd': 0.000982971006274194,
#                  'peak': 10,
#                 }



estimator = MXNet(entry_point='run.py', 
                  source_dir=source_dir,
                  role=role, 
                  train_instance_count=1, 
                  train_instance_type=train_instance_type,
                  framework_version='1.1.0',
                  hyperparameters=hyperparameters,
                  )




In [ ]:
# estimator.fit(inputs)  #  Uncomment this cell if you wish to perform a single run. 
#                        #  For a tuning job, proceed to next cell. 

In [ ]:
# An example of hyperparameters and ranges that you would like to optimize.
hyperparameter_ranges = {
                         'lr': ContinuousParameter(0.1, 0.65),
                         'momentum': ContinuousParameter(0.87, 0.92),    
                         'wd': ContinuousParameter(0.0005, 0.0017),
                        }


metric_definitions = [{'Name': 'Validation-accuracy',   
                       'Regex': 'Validation accuracy=([0-9\\.]+)'}] # We can are the validation accuracy. 

objective_metric_name = 'Validation-accuracy'
tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=120,
                            max_parallel_jobs=2,
                           base_tuning_job_name='my-tuning-job')

In [ ]:
tuner.fit(inputs) # Launch the tuning job!

## The rest of the code is used for visualizing your tuning job only

In [ ]:
import boto3
import sagemaker
import os

region = boto3.Session().region_name
sage_client = boto3.Session().client('sagemaker')


#### Enter you tuning job name
tuning_job_name = 'my-tuning-job'
####


tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuning_job_name)

status = tuning_job_result['HyperParameterTuningJobStatus']
if status != 'Completed':
    print('Reminder: the tuning job has not been completed.')
    
job_count = tuning_job_result['TrainingJobStatusCounters']['Completed']
print("%d training jobs have completed" % job_count)
    
is_minimize = (tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['Type'] != 'Maximize')
objective_name = tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['MetricName']


import pandas as pd

tuner = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)
full_df = tuner.dataframe()

if len(full_df) > 0:
    df = full_df[full_df['FinalObjectiveValue'] > -float('inf')]
    if len(df) > 0:
        df = df.sort_values('FinalObjectiveValue', ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest":min(df['FinalObjectiveValue']),"highest": max(df['FinalObjectiveValue'])})
        pd.set_option('display.max_colwidth', -1)  # Don't truncate TrainingJobName        
    else:
        print("No training jobs have reported valid results yet.")

In [ ]:
import bokeh
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

class HoverHelper():

    def __init__(self, tuning_analytics):
        self.tuner = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.tuner.tuning_ranges.keys():
            tooltips.append( (k, "@{%s}" % k) )

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools='pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset'):
        return [self.hovertool(), standard_tools]

hover = HoverHelper(tuner)

p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type='datetime')
p.circle(source=df, x='TrainingStartTime', y='FinalObjectiveValue')
show(p)

In [ ]:
ranges = tuner.tuning_ranges
figures = []
for hp_name, hp_range in ranges.items():
    categorical_args = {}
    if hp_range.get('Values'):
        # This is marked as categorical.  Check if all options are actually numbers.
        def is_num(x):
            try:
                float(x)
                return 1
            except:
                return 0           
        vals = hp_range['Values']
        if sum([is_num(x) for x in vals]) == len(vals):
            # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
            print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
        else:
            # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
            categorical_args['x_range'] = vals

    # Now plot it
    p = figure(plot_width=500, plot_height=500, 
               title="Objective vs %s" % hp_name,
               tools=hover.tools(),
               x_axis_label=hp_name, y_axis_label=objective_name,
               **categorical_args)
    p.circle(source=df, x=hp_name, y='FinalObjectiveValue')
    figures.append(p)
show(bokeh.layouts.Column(*figures))